In [ ]:
import gmsh
import os
import import_ipynb
import numpy as np
import matplotlib.pyplot as plt
from fem_pre_processing import read_mesh, create_domain as domain
from fem_processing import gaussian_quadrature, matrices_assembly
from fem_pos_processing import graph_results

# Rectangular Domain $\Omega = [0,1]^2$

# `apply_physics()`

In [ ]:
def apply_physics(FINITE_ELEMENT, mesh_data):
    """
    Adiciona uma nova chave 'source' a cada dicionário em conn_data.
    
    Parâmetros:
    - mesh_data: Dicionário contendo os dados da malha.
    - element_type: Tuple (tipo do elemento, ordem).
    
    Retorna:
    - mesh_data: O dicionário atualizado com a chave 'source' em cada elemento de conn_data.
    """
    # Dictionary with all nodes in the mesh
    cell_data = mesh_data['cell']
    nodes_data = mesh_data['nodes']

    # Função forçante (fonte)
    source_function = lambda x, y: 2 * np.pi**2 * np.sin(np.pi * x) * np.sin(np.pi * y)

    # Get the Gauss points and weights
    gauss_points, _ = gaussian_quadrature.gauss_data(FINITE_ELEMENT)
    
    for key, cell in cell_data.items():
        # Constant material properties
        a = cell['material']['a_constant']
        
        # Adicionar as propriedades dos materiais ao dicionário da célula
        cell['stiffness_term'] = []
        cell['mass_term'] = []
        cell['source'] = []
        
        # Get the global coordinates of the nodes
        aie = [nodes_data[idx]['xg'] for idx in cell['conn']]

        # Adicionar a nova chave 'material' ao dicionário da célula
        for xik in gauss_points: 
            # Coordenadas globais no ponto de Gauss
            xge, yge = matrices_assembly.isomapping_to_global_coordinates(FINITE_ELEMENT, aie, xik)
            
            # Adicionar os termos de rigidez e massa ao dicionário da célula
            cell['stiffness_term'].append(a * np.eye(2))
            cell['mass_term'].append(1)

            # Adicionar a fonte ao dicionário da célula
            cell['source'].append(source_function(xge, yge))
            
        # Verifica tipo de fronteira absorvente (ABC)
        cell_data[key]['abc'] = {'type': None, 'conn_idx': None}

    return mesh_data

# `create_domain()`

In [ ]:
def create_domain(FINITE_ELEMENT, BOUNDARY, MATERIAL, h, auto_save=True, view_mesh=False):
    mesh_data = {}
    type, order = FINITE_ELEMENT
    vertices = [(0, 0, 0), (1, 0, 0), (1, 1, 0), (0, 1, 0)]

    # Define a new model
    gmsh.initialize()
    gmsh.model.add("rectangular_domain")

    # Define the points of the domain.
    point_tags = domain.add_points(vertices, h)

    # Criar linhas para formar as bordas do quadrado
    line_tags = domain.add_lines(point_tags)
    
    # Criar um loop de linha e uma superfície plana
    free_space = domain.add_surface(line_tags)

    # Sinconizar o modelo
    gmsh.model.geo.synchronize()

    # Adicionar grupos físicos para Dim=1
    for i, tag in enumerate([line_tags]):
        gmsh.model.addPhysicalGroup(1, tags=tag, tag=BOUNDARY[i]['tag'], name=BOUNDARY[i]['name'])

    # Adicionar grupos físicos para Dim=2
    gmsh.model.addPhysicalGroup(2, [free_space], tag=MATERIAL[0]['tag'], name=MATERIAL[0]['name'])  
    
    # Gerar a malha 2D
    gmsh.option.setNumber("Mesh.SaveAll", 1)
    gmsh.model.mesh.generate(2)
    gmsh.model.mesh.setOrder(order)

    if view_mesh:
        gmsh.fltk.run()
    
    if auto_save:
        os.makedirs("pre_processing/mesh", exist_ok=True)
        file_path = f"pre_processing/mesh/rectangular_domain_{type}{order}.msh"
        print(f"Malha salva em {file_path}")
        gmsh.write(file_path)
        read_mesh.basic_info()
    
    # Create mesh Structure Data from gmsh
    mesh_data['cell'] = read_mesh.get_cell_data(MATERIAL)
    mesh_data['nodes'] = read_mesh.get_nodes_data(BOUNDARY)
    mesh_data['edges'] = read_mesh.get_edge_data()

    # Apply physics to the problem
    mesh_data = apply_physics(FINITE_ELEMENT, mesh_data)
    
    gmsh.finalize()

    return mesh_data

# `set_analytical_solution()`

In [ ]:
def set_analytical_solution(x, y):
    u = np.sin(np.pi * x) * np.sin(np.pi * y)
    grad_u = np.array([np.pi * np.cos(np.pi * x) * np.sin(np.pi * y),
                       np.pi * np.sin(np.pi * x) * np.cos(np.pi * y)])
    return u, grad_u

# `plot_analytical_solution()`

In [ ]:
def plot_analytical_solution(Npts):
    # Definindo a solução analítica u(x, y) e a função fonte f(x, y)
    u = lambda x, y: np.sin(np.pi * x) * np.sin(np.pi * y)
    f = lambda x, y: 2 * np.pi**2 * np.sin(np.pi * x) * np.sin(np.pi * y)
    
    # Criando uma malha para o domínio [0, 1] x [0, 1]
    x = np.linspace(0, 1, Npts)
    y = np.linspace(0, 1, Npts)
    mesh_x, mes_y = np.meshgrid(x, y)

    # Plotando a solução u(x, y)
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.contourf(mesh_x, mes_y, u(mesh_x, mes_y), cmap='viridis')
    plt.colorbar()
    plt.title(r'Analytical Solution $u(x, y)$')
    plt.xlabel(r'$x$')
    plt.ylabel(r'$y$')

    # Plotando f(x, y)
    plt.subplot(1, 2, 2)
    plt.contourf(mesh_x, mes_y, f(mesh_x, mes_y), cmap='gray')
    plt.colorbar()
    plt.title(r'Source function $f(x, y)$')
    plt.xlabel(r'$x$')
    plt.ylabel(r'$y$')
    plt.tight_layout()
    
    # Salvar a figura
    filepath = graph_results.get_dir(f"pos_processing/pictures/analytical_solution.svg")
    plt.savefig(filepath, format="svg")
    plt.close()
    print(f"Arquivo salvo em: {filepath}")